# Convert  Object Store datapoints - Airbus NationWide - into Visual search Map

In [1]:
import pandas
print(pandas.__version__)

0.24.2


In [2]:
import folium
print(folium.__version__)

0.14.0


In [3]:
import numpy as np 
import pandas as pd 
import folium
from folium.plugins import Search
from functools import reduce
import math
# No need since 28Apr23 - Lat/Lon provided by Airbus
# Convert BNG to LAT/LON
#from bng_latlon import OSGB36toWGS84
#
input_coord_csv_file='data/output/NBS_Portfolio_Bristol_latlon_230216_csv/part-00000-5c3ba62f-e549-448a-bfa0-6d70538207c5-c000.csv'
df_nationwide_loanbook = pd.read_csv(input_coord_csv_file, header = 0 )


In [5]:
df_nationwide_loanbook.describe()

,ID,UPRN,POSTCODE_FLAG,EASTING,NORTHING,RHIGHSCORE,RHIGHSCORE_ADJ,CHIGHSCORE,CHIGHSCORE_ADJ,SWHIGHSCORE,...,RCP45_2050S_DEF_C_RR,RCP85_2050S_UNDEF_F_RR,RCP85_2050S_UNDEF_P_RR,RCP85_2050S_UNDEF_T_RR,RCP85_2050S_UNDEF_C_RR,RCP85_2050S_DEF_F_RR,RCP85_2050S_DEF_T_RR,RCP85_2050S_DEF_C_RR,LATITUDE,LONGITUDE
count,18085.000000,1.808500e+04,18085.000000,18085.000000,18085.000000,18085.000000,18085.00000,18085.000000,18085.00000,18085.000000,...,18075.000000,18075.000000,18075.000000,18075.000000,18075.000000,18075.000000,18075.000000,18075.000000,18085.000000,18085.000000
mean,9043.000000,8.901323e+09,0.002765,363001.261979,174118.554777,0.517556,0.44031,0.370805,0.09953,0.551562,...,4.389986,1.399336,3.404869,1.235076,5.336985,1.399336,0.891286,5.030152,51.464792,-2.534008
std,5220.834145,2.814950e+10,0.052509,5482.263478,5092.701827,2.950690,2.63575,2.569122,1.14896,1.643107,...,15.618402,10.035297,13.124174,10.103181,18.234758,10.035297,7.390518,17.074913,0.045734,0.078986
min,1.000000,1.470000e+02,0.000000,350002.000000,160034.000000,0.000000,0.00000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,51.338307,-2.721267
25%,4522.000000,1.116180e+05,0.000000,359164.950000,170934.000000,0.000000,0.00000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,51.436011,-2.589203
50%,9043.000000,3.731460e+05,0.000000,361955.000000,174209.250000,0.000000,0.00000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,51.465525,-2.549161
75%,13564.000000,5.924060e+05,0.000000,365945.000000,177516.230000,0.000000,0.00000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,51.495229,-2.491613
max,18085.000000,2.000029e+11,1.000000,379995.000000,186975.000000,22.000000,22.00000,20.000000,20.00000,10.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,51.581107,-2.288625


In [6]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_nationwide_loanbook.dtypes)



ID                                    int64
UPRN                                  int64
POSTCODE_FLAG                         int64
EASTING                             float64
NORTHING                            float64
RHIGHSCORE                            int64
RHIGHSCORE_ADJ                        int64
CHIGHSCORE                            int64
CHIGHSCORE_ADJ                        int64
SWHIGHSCORE                           int64
SWHIGHSCORE_ADJ                       int64
GWHIGHSCORE                           int64
COMBINEDSCORE                         int64
COMBINEDSCORE_ADJ                     int64
R_RCP26_2050_UD                     float64
R_RCP26_2050_D                      float64
S_RCP26_2050_UD                     float64
S_RCP26_2050_D                      float64
C_RCP26_2050_UD                     float64
C_RCP26_2050_D                      float64
COMB_RCP26_2050_UD                  float64
COMB_RCP26_2050_D                   float64
R_RCP45_2050_UD                 

In [9]:
# Lambda function apply convertion CURRENTLOAN and LTV to INITIAL_BORROW_CAPITAL
#CURRENTLOAN                         float64
#CURRENTLTV                     float64

vars_ln=['CURRENTLOAN','CURRENTLTV']
cuurlnamt='CURRENTLOAN'
currltv='CURRENTLTV'
df_nationwide_loanbook['INITIAL_BORROW_CAPITAL'] = df_nationwide_loanbook[vars_ln].apply(lambda x : x[cuurlnamt]*1/x[currltv] ,axis=1)


In [10]:
df_nationwide_loanbook.dtypes

ID                          int64
UPRN                        int64
POSTCODE_FLAG               int64
EASTING                   float64
NORTHING                  float64
RHIGHSCORE                  int64
RHIGHSCORE_ADJ              int64
CHIGHSCORE                  int64
CHIGHSCORE_ADJ              int64
SWHIGHSCORE                 int64
SWHIGHSCORE_ADJ             int64
GWHIGHSCORE                 int64
COMBINEDSCORE               int64
COMBINEDSCORE_ADJ           int64
R_RCP26_2050_UD           float64
R_RCP26_2050_D            float64
S_RCP26_2050_UD           float64
S_RCP26_2050_D            float64
C_RCP26_2050_UD           float64
C_RCP26_2050_D            float64
COMB_RCP26_2050_UD        float64
COMB_RCP26_2050_D         float64
R_RCP45_2050_UD           float64
R_RCP45_2050_D            float64
S_RCP45_2050_UD           float64
S_RCP45_2050_D            float64
C_RCP45_2050_UD           float64
C_RCP45_2050_D            float64
COMB_RCP45_2050_UD        float64
COMB_RCP45_205

In [11]:
df_nationwide_loanbook.head(3)

,ID,UPRN,POSTCODE_FLAG,EASTING,NORTHING,RHIGHSCORE,RHIGHSCORE_ADJ,CHIGHSCORE,CHIGHSCORE_ADJ,SWHIGHSCORE,...,RCP85_2050S_UNDEF_F_RR,RCP85_2050S_UNDEF_P_RR,RCP85_2050S_UNDEF_T_RR,RCP85_2050S_UNDEF_C_RR,RCP85_2050S_DEF_F_RR,RCP85_2050S_DEF_T_RR,RCP85_2050S_DEF_C_RR,LATITUDE,LONGITUDE,INITIAL_BORROW_CAPITAL
0,1,147,0,358754.72,179718.88,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,51.514877,-2.595776,282617.0
1,2,148,0,358946.00,179716.00,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,51.514865,-2.593020,291961.0
2,7,155,0,358974.00,179709.00,0,0,0,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,51.514804,-2.592615,456431.0


In [12]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_nationwide_loanbook.head(3))

   ID  UPRN  POSTCODE_FLAG    EASTING   NORTHING  RHIGHSCORE  RHIGHSCORE_ADJ  \
0   1   147              0  358754.72  179718.88           0               0   
1   2   148              0  358946.00  179716.00           0               0   
2   7   155              0  358974.00  179709.00           0               0   

   CHIGHSCORE  CHIGHSCORE_ADJ  SWHIGHSCORE  SWHIGHSCORE_ADJ  GWHIGHSCORE  \
0           0               0            0                0            0   
1           0               0            0                0            0   
2           0               0            2                2            0   

   COMBINEDSCORE  COMBINEDSCORE_ADJ  R_RCP26_2050_UD  R_RCP26_2050_D  \
0              0                  0              0.0             0.0   
1              0                  0              0.0             0.0   
2              2                  2              0.0             0.0   

   S_RCP26_2050_UD  S_RCP26_2050_D  C_RCP26_2050_UD  C_RCP26_2050_D  \
0             

In [ ]:
### Formula ABOVE

In [ ]:
exit()